## **Business Problem**

This data-driven project aims to derive actionable insights to enhance customer retention by focusing on high-value customers and identifying churn risks. The primary concern is the alarming customer churn rate faced by the company, leading to significant revenue loss. A higher churn rate indicates a critical issue that requires an in-depth analysis to uncover the underlying causes and patterns.

To better understand customer churn in the Telecom Company, a detailed analytical approach will be used.

First, a comprehensive examination of the dataset will shed light on factors such as customer demographics, service tenure, billing patterns, common subscriptions, preferred offers, and additional services demanded. Along with this, data visualization will play a key role. By using various graphs and charts, we can better understand the reasons behind the higher churn rates. These visualizations will meticulously highlight trends within the dataset for different customer segments, helping stakeholders to better understand customer behavior.

Lastly, to ensure that our findings are grounded in solid evidence, we will apply statistical tools, including t-tests and regression analyses, to validate our insights and further explore the factors driving customer churn.

In [3]:
import sys
print(sys.version)

3.12.1 (main, Sep 30 2024, 17:05:21) [GCC 9.4.0]


In [4]:
# Data Loading and Manipulation modules
import pandas as pd
import numpy as np

# Visualization Modules and set the 'ggplot' style
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Load the dataset
df = pd.read_csv('Customer Churn.csv')

# Displaying the first few rows of the dataset to verify loading
df.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability
